In [ ]:
pip install -r ../requirements.txt --quiet

In [17]:
import json
from tqdm import tqdm
from razdel import sentenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartTokenizer, MBartForConditionalGeneration
import torch

In [20]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
model_name = "cointegrated/rut5-base-absum" #"sarahai/ru-sum" 

In [26]:
# для "sarahai/ru-sum" 
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
# model.eval()

# для cointegrated/rut5-base-absum
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
# model.eval()


# для "cointegrated/rut5-base-absum"
model_name = "cointegrated/rut5-base-absum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [27]:
# def abstractive_summary(text, max_symbols=300):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     summary_ids = model.generate(
#           **inputs,
#           max_length=100,
#           min_length=60,
#           num_beams=30,
#           repetition_penalty=3.0,
#           length_penalty=1.2,
#           no_repeat_ngram_size=10,
#           early_stopping=True
#       )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary[:max_symbols]


def improved_abstractive_summary(text, max_symbols=300, device='cuda'):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to('mps')
    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=100,          # подогнал под размер 300 на глаз
            min_length=60,
            num_beams=30,            # взял прямо жирно, но так мне больше нравится
            temperature=0.9,         # Для чуть большей вариативности
            repetition_penalty=1.5,  # Мягче штрафуем повторения
            length_penalty=1.0,      # Нейтральное влияние на длину
            no_repeat_ngram_size=3,  
            early_stopping=True,
            do_sample=True          
        )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary[:max_symbols]


In [28]:
with open("ex2.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

summaries = []
for text in tqdm(texts, desc="Summarizing texts"):
    abs_summary = improved_abstractive_summary(text)
    summaries.append(abs_summary)

with open("summaries.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)

Summarizing texts: 100%|██████████| 5/5 [00:43<00:00,  8.71s/it]


In [24]:
reference_summaries = [
    "В избушке живёт пёс Дик, который любит наблюдать, как курят. Он обжорлив и любит есть рыбью требуху. Однажды автор собрал чернику на болоте и пытался научить Дика уму, показывая, как есть ягоды с куста. Через два дня пёс собрал чернику вокруг избушки, чему автор был рад.",
    "Скворец Петруша любит подражать голосам и звукам. Поэт сочинял стихи и печатал их в журналах. В песне Петруши есть трели, свисты и рулады, которые звучат как весенние природные явления, принося радость и настроение весны.",
    "Зайцы обычно не собирают букеты, ведь у каждого из них есть свой цветок — хвост. Но один заяц собрал настоящий букет и не знает, кому его подарить. Лисе и волку цветы не нужны, мишка любит ягоды, а барсук может даже наброситься. Решают отдать букет барсуку и посмотреть, что будет.",
    "Ночью у костра главного героя охватывает страх. Ему кажется, что кто-то наблюдает из темноты и шепчется вокруг. После громкого крика появляется шум листьев — это шевеление листобоя в кронах деревьев. Рассказ передает атмосферу тревоги и таинственности леса.",
    "В ночь в печной трубе задул листобой — холодный октябрьский ветер. Форточка была уже раскрыта настежь и полна берёзовых листьев. Листья сбрасывала берёза под окном, а сам хозяин уже скрылся куда-то."
]

for i in range(len(texts)):
    print(f"Длинна исходного текста: {len(texts[i])}, Длинна реферата: {len(summaries[i])}, Длинна референса: {len(reference_summaries[i])}")

Длинна исходного текста: 1054, Длинна реферата: 171, Длинна референса: 271
Длинна исходного текста: 1101, Длинна реферата: 179, Длинна референса: 221
Длинна исходного текста: 807, Длинна реферата: 191, Длинна референса: 281
Длинна исходного текста: 684, Длинна реферата: 150, Длинна референса: 257
Длинна исходного текста: 740, Длинна реферата: 200, Длинна референса: 198


In [29]:
from rouge import Rouge

rouge = Rouge()

for i, (summary, reference) in enumerate(zip(summaries, reference_summaries)):
    score = rouge.get_scores(summary, reference, avg=False)
    print(f"\nРезультаты для пары {i+1}:")
    print(f"ROUGE-1: {score[0]['rouge-1']}")
    print(f"ROUGE-2: {score[0]['rouge-2']}")
    print(f"ROUGE-L: {score[0]['rouge-l']}")


Результаты для пары 1:
ROUGE-1: {'r': 0.3157894736842105, 'p': 0.4, 'f': 0.35294117153979243}
ROUGE-2: {'r': 0.09090909090909091, 'p': 0.13333333333333333, 'f': 0.10810810328707107}
ROUGE-L: {'r': 0.3157894736842105, 'p': 0.4, 'f': 0.35294117153979243}

Результаты для пары 2:
ROUGE-1: {'r': 0.45161290322580644, 'p': 0.5185185185185185, 'f': 0.4827586157134364}
ROUGE-2: {'r': 0.3333333333333333, 'p': 0.3793103448275862, 'f': 0.354838704698231}
ROUGE-L: {'r': 0.45161290322580644, 'p': 0.5185185185185185, 'f': 0.4827586157134364}

Результаты для пары 3:
ROUGE-1: {'r': 0.22727272727272727, 'p': 0.3225806451612903, 'f': 0.2666666618168889}
ROUGE-2: {'r': 0.0625, 'p': 0.0967741935483871, 'f': 0.07594936232014131}
ROUGE-L: {'r': 0.22727272727272727, 'p': 0.3225806451612903, 'f': 0.2666666618168889}

Результаты для пары 4:
ROUGE-1: {'r': 0.16666666666666666, 'p': 0.23076923076923078, 'f': 0.19354838222684714}
ROUGE-2: {'r': 0.027777777777777776, 'p': 0.03571428571428571, 'f': 0.03124999507812